In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install concrete-ml

In [ ]:
import opendatasets as od
# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/naveengowda16/logistic-regression-heart-disease-prediction/data'
# Using opendatasets let's download the data sets
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gregorypachisuoc
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/naveengowda16/logistic-regression-heart-disease-prediction


100%|██████████| 58.4k/58.4k [00:00<00:00, 51.4MB/s]

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

from concrete.ml.quantization.quantized_module import QuantizedModule
from concrete.ml.torch.compile import compile_brevitas_qat_model

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
df=pd.read_csv('/content/logistic-regression-heart-disease-prediction/framingham_heart_disease.csv')

In [ ]:
df.fillna(df.mean(), inplace=True)

count_final_column1 = (df.iloc[:, -1] == 1).sum()
print("Count of samples with 1 in the final column:", count_final_column1)

count_final_column = (df.iloc[:, -1] == 0).sum()
print("Count of samples with 0 in the final column:", count_final_column)

Count of samples with 1 in the final column: 644
Count of samples with 0 in the final column: 3594


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
cols_to_be_normalized = ["age","cigsPerDay","totChol","sysBP","diaBP","BMI","heartRate","glucose"]
cols_not_to_be_normalized = ["male","currentSmoker","BPMeds","prevalentStroke","prevalentHyp","diabetes","TenYearCHD"]
data=df
normalized = normalize(data[cols_to_be_normalized])
boolean = data[cols_not_to_be_normalized]
df_normalized = pd.DataFrame(normalized,columns = cols_to_be_normalized)
df_boolean = pd.DataFrame(boolean,columns=cols_not_to_be_normalized)
# df_final = pd.concat([df_normalized,df_boolean],axis = 1)
# df_final = df_normalized.join(df_boolean)
df_final = df_normalized.merge(df_boolean,left_index=True, right_index=True)

df_final.head(20)

,age,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,male,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,TenYearCHD
0,0.148753,0.000000,0.743764,0.404302,0.266992,0.102868,0.305134,0.293691,1,0,0.0,0,0,0,0
1,0.144425,0.000000,0.784921,0.379902,0.254314,0.090203,0.298270,0.238616,0,0,0.0,0,0,0,0
2,0.154476,0.064365,0.788469,0.410326,0.257459,0.081550,0.241368,0.225277,1,1,0.0,0,0,0,0
3,0.190605,0.093740,0.703050,0.468700,0.296843,0.089303,0.203103,0.321841,0,1,0.0,0,1,0,1
4,0.131263,0.065631,0.813259,0.370960,0.239697,0.065917,0.242551,0.242551,0,1,0.0,0,0,0,0
5,0.126811,0.000000,0.672391,0.530835,0.324399,0.089357,0.227079,0.291959,0,0,0.0,0,1,0,0
6,0.220005,0.000000,0.715890,0.481916,0.247942,0.115625,0.209529,0.296832,0,0,0.0,0,0,0,1
7,0.125666,0.055852,0.874077,0.279258,0.198273,0.060543,0.220614,0.217821,0,1,0.0,0,0,0,0
8,0.156107,0.000000,0.780534,0.424791,0.267183,0.079134,0.228156,0.237162,1,0,0.0,0,1,0,0
9,0.130834,0.091280,0.684598,0.492911,0.325564,0.071837,0.282967,0.267754,1,1,0.0,0,1,0,0


In [ ]:
import numpy as np
X = df_final.drop("TenYearCHD",axis = 1)
Y = df_final["TenYearCHD"]
X = np.array(X)
Y = np.array(Y)
X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state = 1,test_size = 0.2)

In [ ]:
# pylint: disable-next=too-many-arguments


def train(
    torch_model,
    X_train,
    X_test,
    y_train,
    y_test,
    criterion,
    optimizer,
    epochs=10,
    batch_size=1,
    shuffle=True,
    device="cpu",
):
    X_train = torch.tensor(X_train).float()
    X_test = torch.tensor(X_test).float()
    y_train = torch.tensor(y_train)

    train_loader = DataLoader(
        TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=shuffle
    )
    torch_model.train()
    for epoch in range(epochs):
        total_loss = []
        y_pred_all = []
        y_true_all = []

        for batch_index, (X_batch, y_batch) in enumerate(train_loader):
            # Forward pass
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = torch_model(X_batch)
            y_pred_all.append(y_pred.argmax(1).detach().cpu().numpy())
            y_true_all.append(y_batch.detach().cpu().numpy())

            # Compute loss
            loss = criterion(y_pred, y_batch)
            if torch.isnan(loss):
                print("y_pred", y_pred)
                print("y_batch", y_batch)
                raise ValueError(f"Loss diverged at step: {batch_index}")

            # Backward pass
            optimizer.zero_grad()
            loss.backward()

            # Update weights
            optimizer.step()

            total_loss.append(loss.cpu().item())

        # Print epoch number, loss and accuracy
        y_pred_all = numpy.concatenate(y_pred_all)
        y_true_all = numpy.concatenate(y_true_all)
        accuracy = numpy.mean(y_pred_all == y_true_all)
        print(
            f"Epoch: {epoch:02} | Loss: {numpy.mean(total_loss):.4f} |"
            f" Train Accuracy: {100*accuracy:.2f}%"
        )

    # Compute test accuracy once training is done
    torch_model.eval()
    fp32_pred = torch_model(X_test.to(device)).cpu().argmax(1).float().detach().numpy()
    accuracy = numpy.mean(fp32_pred == y_test)
    print(f"\nTest Accuracy Fp32: {accuracy*100:.2f}%")

    return accuracy

In [ ]:
def test_in_fhe(quantized_numpy_module, X_test, y_test, simulate=True):
    if not simulate:
        print("Generating key")
        start_key = time.time()
        quantized_numpy_module.fhe_circuit.keygen()
        end_key = time.time()
        print(f"Key generation finished in {end_key - start_key:.2f} seconds")

    fhe_mode = "simulate" if simulate else "execute"

    start_infer = time.time()
    predictions = quantized_numpy_module.forward(X_test, fhe=fhe_mode).argmax(1)
    end_infer = time.time()

    if not simulate:
        print(
            f"Inferences finished in {end_infer - start_infer:.2f} seconds "
            f"({(end_infer - start_infer)/len(X_test):.2f} seconds/sample)"
        )

    # Compute accuracy
    accuracy = numpy.mean(predictions == y_test) * 100
    print(
        "FHE " + ("(simulation) " * simulate) + f"accuracy: {accuracy:.2f}% on "
        f"{len(X_test)} examples."
    )
    return predictions

In [ ]:
import brevitas.nn as qnn
from brevitas.core.bit_width import BitWidthImplType
from brevitas.core.quant import QuantType
from brevitas.core.restrict_val import FloatToIntImplType, RestrictValueType
from brevitas.core.scaling import ScalingImplType
from brevitas.core.zero_point import ZeroZeroPoint
from brevitas.inject import ExtendedInjector
from brevitas.quant.solver import ActQuantSolver, WeightQuantSolver
from dependencies import value
from torch.nn.utils import prune


# More details on injectors at
# https://github.com/Xilinx/brevitas/blob/master/ARCHITECTURE.md#injectors-and-quantizers
class CommonQuant(ExtendedInjector):
    bit_width_impl_type = BitWidthImplType.CONST
    scaling_impl_type = ScalingImplType.CONST
    restrict_scaling_type = RestrictValueType.FP
    zero_point_impl = ZeroZeroPoint
    float_to_int_impl_type = FloatToIntImplType.ROUND
    scaling_per_output_channel = False
    narrow_range = True
    signed = True

    @value
    def quant_type(bit_width):  # pylint: disable=no-self-argument
        if bit_width is None:
            return QuantType.FP
        if bit_width == 1:
            return QuantType.BINARY
        return QuantType.INT


class CommonWeightQuant(CommonQuant, WeightQuantSolver):  # pylint: disable=too-many-ancestors
    scaling_const = 1.0
    signed = True


class CommonActQuant(CommonQuant, ActQuantSolver):  # pylint: disable=too-many-ancestors
    min_val = -1.0
    max_val = 1.0

In [ ]:
class QATPrunedSimpleNet(nn.Module):
    def __init__(self, n_hidden, qlinear_args, qidentity_args):
        super().__init__()

        self.pruned_layers = set()

        self.quant_inp = qnn.QuantIdentity(**qidentity_args)

        self.fc1 = qnn.QuantLinear(IN_FEAT, n_hidden, **qlinear_args)

        self.relu1 = qnn.QuantReLU(bit_width=qidentity_args["bit_width"])

        self.fc2 = qnn.QuantLinear(n_hidden, n_hidden, **qlinear_args)

        self.relu2 = qnn.QuantReLU(bit_width=qidentity_args["bit_width"])

        self.fc3 = qnn.QuantLinear(n_hidden, OUT_FEAT, **qlinear_args)

        for m in self.modules():
            if isinstance(m, qnn.QuantLinear):
                torch.nn.init.uniform_(m.weight.data, -1, 1)

    def forward(self, x):
        x = self.quant_inp(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

    def prune(self, max_non_zero):
        # Linear layer weight has dimensions NumOutputs x NumInputs
        for name, layer in self.named_modules():
            if isinstance(layer, qnn.QuantLinear):
                num_zero_weights = (layer.weight.shape[1] - max_non_zero) * layer.weight.shape[0]
                if num_zero_weights <= 0:
                    continue
                print(f"Pruning layer {name} factor {num_zero_weights}")
                prune.l1_unstructured(layer, "weight", amount=num_zero_weights)
                self.pruned_layers.add(name)

    def unprune(self):
        for name, layer in self.named_modules():
            if name in self.pruned_layers:
                prune.remove(layer, "weight")
                self.pruned_layers.remove(name)

In [ ]:
IN_FEAT=14
OUT_FEAT=2
# Add MPS (for macOS with Apple Silicon or AMD GPUs) support when error is fixed. For now, we
# observe a decrease in torch's top1 accuracy when using MPS devices
# FIXME: https://github.com/zama-ai/concrete-ml-internal/issues/3953
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define the batch size
batch_size = 1
n_epochs = 7
n_hidden = 100

# We use 100 neurons with only 20 that will be active. Having many neurons
# out of which we chose the best ones increases the robustness of training
# while keeping the accumulator size low
torch_model = QATPrunedSimpleNet(
    n_hidden=n_hidden,
    qlinear_args={
        "weight_bit_width": 3,
        "weight_quant": CommonWeightQuant,
        "bias": True,
        "bias_quant": None,
        "narrow_range": True,
    },
    qidentity_args={"bit_width": 3, "act_quant": CommonActQuant},
)
torch_model.prune(20)

Pruning layer fc2 factor 8000
Pruning layer fc3 factor 160


In [ ]:
torch_model = torch_model.to(device)
optimizer = torch.optim.AdamW(torch_model.parameters(), lr=0.001)
accuracy = train(
    torch_model,
    X_train,
    X_test,
    y_train,
    y_test,
    criterion,
    optimizer,
    epochs=n_epochs,
    batch_size=batch_size,
    device=device,
)
torch_model.unprune()

Epoch: 00 | Loss: 0.6845 | Train Accuracy: 79.53%
Epoch: 01 | Loss: 0.5336 | Train Accuracy: 81.59%
Epoch: 02 | Loss: 0.4770 | Train Accuracy: 82.92%
Epoch: 03 | Loss: 0.4560 | Train Accuracy: 82.89%
Epoch: 04 | Loss: 0.4328 | Train Accuracy: 83.51%
Epoch: 05 | Loss: 0.4301 | Train Accuracy: 84.01%
Epoch: 06 | Loss: 0.4145 | Train Accuracy: 84.40%

Test Accuracy Fp32: 85.61%
